# Chapter 4 - Model Development of An Open-loop PI Controller

## Introduction

> [Development - Models](https://docs.andes.app/en/latest/modeling/models.html#models). The terminology "model" is used to describe the mathematical representation of a type of device, such as synchronous generators or turbine governors. The terminology "device" is used to describe a particular instance of a model, for example, a specific generator. To define a model in ANDES, two classes, ``ModelData`` and ``Model`` need to be utilized. Class ``ModelData`` is used for defining parameters that will be provided from input files. It provides API for adding data from devices and managing the data. Class ``Model`` is used for defining other non-input parameters, service variables, and DAE variables. It provides API for converting symbolic equations, storing Jacobian patterns, and updating equations.

In this chapter, you will learn to develop a dynamic model in ANDES.

## Objective

In this chapter, you will:
- Go through the model development workflow
- Develop an open-loop PI controller

## Submission

After complete the chapter, please save and submit this jupyter notebook file in **CANVAS** with **FORMATTED** name:

`FirstName_LastName_NetID_ChID.ipynb`, for example, `Tim_Cook_tcook3_Ch4.ipynb`.

# Code Server

You will use the blue "+" button to start a "Code Server" session in the
browser. If you ever used VS Code, this is a modified version of VS Code. Using
this online version will save you some time to configure the environment. 

You will use the top menu button to open the project folder. Select "File - Open
Folder" and open `/home/jovyan/repos/andes`. Code Server will reload, you will
need to Trust the Author of the notebook to run the code.


## Model Development

In this chapter, you will use the existing ``block`` [PIController](https://docs.andes.app/en/latest/modeling/block.html#pi-controllers) in ANDES to develop the ``model`` open loop PI controller.
You need to read the [examples of modeling](https://docs.andes.app/en/stable/modeling/examples.html) and then finish the exercise below.

### Model Design

The model ``OLPI`` is an open-loop PI controller that takes Generator speed deviation `wd` as input.
The block diagrams is shown below:
```
     ┌────────────────────┐
     │      ki     skd    │
u -> │kp + ─── + ───────  │ -> y
     │      s    1 + sTd  │
     └────────────────────┘
```

### Working Branch

To keep a tidy work environment, you need to create a **git branch** named ``olpi`` from the most recent **develop** branch by command: ``git checkout -b olpi develop``.
The model development should be done in the ``olpi`` branch.

### Model Implementation

You will need to copy the provided `olpi.py` to the directory ``$HOME/repos/andes/andes/models/experimental/olpi.py``.

The model ``OLPI`` is a PI controller with gain ``kP`` and integral constant ``kI``. You can take advantage of the existing ``block`` [PIDController](https://docs.andes.app/en/latest/modeling/block.html#pi-controllers).

In the file ``olpi.py``, two classes, ``OLPIData`` and ``OLPIModel`` are used.

#### OLPIData

```python
class OLPIData(ModelData):
    """
    Data for open-loop PI controller..
    """

    def __init__(self):
        super().__init__()
        self.gov = IdxParam(model='TurbineGov',
                            info='Turbine governor idx',
                            mandatory=True,
                            )
        self.kP = NumParam(info='PID proportional coeff.',
                           tex_name='k_P',
                           default=1,
                           )
        self.kI = NumParam(info='PID integrative coeff.',
                           tex_name='k_I',
                           default=1,
                           )
        self.kD = NumParam(info='PID derivative coeff.',
                           tex_name='k_D',
                           default=0,
                           )
        self.tD = NumParam(info='PID derivative time constant coeff.',
                           tex_name='t_D',
                           default=0,
                           )
```
``gov`` is ``IdxParam`` storing ``idx`` of other models.
``kP``, ``kI``, ``kD``, and ``tD`` are ``NumParam`` storing the proportional gain and integral gain, respectively.

There are other types *Prameters* that not included in this model, such as ``DataParam``, ``ExtParam``, and ``TimerParam``. You can find more details in [Development - Parameters](https://docs.andes.app/en/latest/modeling/parameters.html).

#### OLPIModel

```python
class OLPIModel(Model):
    """
    Implementation for open-loop PI controller.
    """

    def __init__(self, system, config):
        Model.__init__(self, system, config)
        self.group = 'Experimental'
        self.flags.tds = True

        self.wd = ExtAlgeb(model='TurbineGov', src='pout', indexer=self.gov,
                           info='Generator speed deviation',
                           unit='p.u.',
                           tex_name=r'\omega_{dev}',
                           )
        self.pout = ExtAlgeb(model='TurbineGov', src='pout', indexer=self.gov,
                             tex_name='P_{out}',
                             info='Turbine governor output',
                             )
        self.pout0 = ConstService(v_str='pout',
                                  tex_name='P_{out0}',
                                  info='initial turbine governor output',
                                  )
        self.PID = PIDController(u=self.wd, kp=self.kP, ki=self.kI,
                                 kd=self.kD, Td=self.tD,
                                 tex_name='PID', info='PID', name='PID',
                                 ref=self.pout0,
                                 )
```
``wd`` is a ``ExtAlgeb`` that represents Generator speed deviation.
``pout`` is a ``ExtAlgeb`` that represents the turbine governor output.
``pout0`` is a ``ConstService`` that stores the initial turbine governor output.
``PID`` is a block ``PIDController`` that takes ``pout`` as input.

``ExtAlgeb`` and ``ExtState`` are the bridges that connects different models. They take an input parameter ``indexer`` to identify the target model.

There are other types *Variables* that not included in this model, such as ``State``, ``Algeb``, and ``ExtState``. You can find more details in [Development - Variables](https://docs.andes.app/en/latest/modeling/variables.html).

Besides, *Services*, *Discrete*, and *Blocks* are also used in the model. You can find more details in [Development](https://docs.andes.app/en/latest/modeling/index.html).

#### Model Finalization

Then, you can assemble ``OLPIData`` and ``OLPIModel`` into ``OLPI``.
```python
class OLPI(OLPIData, OLPIModel):
    r"""
    Open-loop PI controller that takes Generator speed deviation as input.

    ```
        ┌────────────────────┐
        │      ki     skd    │
    u ->│kp + ─── + ───────  │ -> y
        │      s    1 + sTd  │
        └────────────────────┘
    ```
    """

    def __init__(self, system, config):
        OLPIData.__init__(self)
        OLPIModel.__init__(self, system, config)
```

In ``andes/models/__init__.py``, ``file_classes`` holds the class names to be imported. In the line `experiemntal`, you can add the newly developed model ``OLPI`` into it.

### Model Test

After the code implementation, the model ``OLPI`` needs to be tested.

Test case ``ieee14_olpi.xlsx`` is revised from [ieee14/ieee14_ace.xlsx](https://github.com/cuihantao/andes/blob/master/andes/cases/ieee14/ieee14_ace.xlsx).

Re-generate the pycode by run ``andes prep`` in terminal. Sometimes there occur multi-functionalities, where you may need `andes prep -f` to re-generate all model code.

Load the test case, and run the ``ss.TDS.init()`` to see if the initialization is successful.

In [ ]:
import andes
andes.config_logger(stream_level=20)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ss = andes.run('./case/ieee14_olpi.xlsx',
               default_config=True,
               no_output=True,
               setup=False)

In [ ]:
ss.add('Toggle', dict(model="GENROU", dev='GENROU_5', t=1.0))

In [ ]:
ss.setup()

In [ ]:
ss.OLPI.as_df()

In [ ]:
ss.PFlow.run()

In [ ]:
ss_init = ss.TDS.init()

In [ ]:
ss.TDS.config.tf = 20
ss.TDS.config.criteria = 0
ss.TDS.run()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

ss.TDS.plt.plot(ss.TGOV1.pout,
                fig=fig, ax=ax[0, 0], show=False,
                grid=True,
                title='Turbine Governor Output Power',
                ylabel='Active Power [p.u.]',
                )
ss.TDS.plt.plot(ss.GENROU.omega,
                fig=fig, ax=ax[0, 1], show=False,
                grid=True, ytimes=60,
                title='Generator Frequency',
                ylabel='Frequency [Hz]',
                )
ss.TDS.plt.plot(ss.OLPI.PID.u,
                legend=False,
                hline=[ss.OLPI.PID.ref.v],
                fig=fig, ax=ax[1, 0], show=False,
                grid=True,
                title='OLPI Input',
                )
ss.TDS.plt.plot(ss.OLPI.PID_y,
                legend=False,
                fig=fig, ax=ax[1, 1], show=False,
                grid=True,
                title='OLPI Output',
                )

From the above figure, it can be seen that ``OLPI`` can response to the input signal correctly.